In [1]:
import os
from pathlib import Path

NOTEBOOK_PATH: Path = Path(os.path.abspath(''))

DATA_PATH: Path = NOTEBOOK_PATH.parent.joinpath('data')

PATH_FOR_RESULT: Path = NOTEBOOK_PATH.joinpath('RESULT')

if not PATH_FOR_RESULT.exists():
    PATH_FOR_RESULT.mkdir()

print(f'当前运行目录：{NOTEBOOK_PATH}')
print(f'当前数据目录：{DATA_PATH}')

当前运行目录：D:\Development\Python\InvestmentWorkshop\notebook
当前数据目录：D:\Development\Python\InvestmentWorkshop\data


# 尝试输出至文件

## 准备

### 引用模块

In [2]:
import datetime as dt

import pandas as pd

from utility import (
    get_available_datafile_name,
    load_csv_as_dataframe,
    get_saved_filename,
    save_dataframe_to_csv
)
from InvestmentWorkshop.indicator.chan import (
    theory_of_chan_2,
    plot_theory_of_chan_2,
)

Load succeed.


### 设置 pandas

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

### 可用数据

In [4]:
available_datafile = get_available_datafile_name(DATA_PATH, 'merged')
for symbol, _ in available_datafile:
    print(f'symbol: {symbol:>12}')

symbol:    DCE.c2201
symbol:    DCE.i2201
symbol:  SHFE.ag2106
symbol:  SHFE.al2111
symbol:  SHFE.al2112
symbol:  SHFE.hc2110
symbol:  SHFE.hc2201
symbol:  SHFE.hc2202
symbol:  SHFE.rb2201


### 声明数据

In [5]:
symbol: str = 'SHFE.al2112'
period: str = 'Minute'
dt_start: str = '2021-10-15 13:30:00'
dt_end: str = '2021-10-19 00:59:00'

### 加载数据

In [6]:
origin_data: pd.DataFrame = load_csv_as_dataframe(DATA_PATH.joinpath(f'{symbol}_{period}.csv'))
df_ohlc: pd.DataFrame = origin_data.loc[dt_start:dt_end, ['open', 'high', 'low', 'close']]

df_ohlc.info()
print('-' * 100)
df_ohlc.head(5)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 795 entries, 2021-10-15 13:30:00 to 2021-10-19 00:59:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    795 non-null    float64
 1   high    795 non-null    float64
 2   low     795 non-null    float64
 3   close   795 non-null    float64
dtypes: float64(4)
memory usage: 31.1 KB
----------------------------------------------------------------------------------------------------


,open,high,low,close
datetime,,,,
2021-10-15 13:30:00,23900.0,23900.0,23840.0,23845.0
2021-10-15 13:31:00,23845.0,23895.0,23845.0,23895.0
2021-10-15 13:32:00,23895.0,23895.0,23855.0,23875.0
2021-10-15 13:33:00,23875.0,23895.0,23840.0,23845.0
2021-10-15 13:34:00,23845.0,23855.0,23830.0,23830.0


## 计算

In [7]:
count: int = 400
merged_candle_list, fractal_list = theory_of_chan_2(df_ohlc, count, debug=False)

## 绘图

### 文件名

In [8]:
# 图片设定
width: int = 6400
height: int = 4800
dpi: int = 288

# 时间
now: str = dt.datetime.now().isoformat()
now = now.replace(':', '-').replace(' ', '_')

# 目录名模板
pattern_directory: str = f'{symbol}_{width}X{height}@{dpi}dpi_{now}'

directory: Path = PATH_FOR_RESULT.joinpath(pattern_directory)
if not directory.exists():
    directory.mkdir()

# 文件名模板
pattern = '{symbol}_{i}_of_{n}.png'

# 每张图多少根K线
count: int = 600
# 多少重复
duplicated: int = 60

# 第1张图，0 ~ 800
# 第2张图，800-60 ~ 800*2-60
# 第3张图，800*2-60 ~ 800*3-60

n: int = len(df_ohlc) // count
if len(df_ohlc) % n > 0:
    n += 1

time_start = dt.datetime.now()
for i in range(n):
    filename: Path = directory.joinpath(
        pattern.format(symbol=symbol, i=i+1, n=n)
    )
    start = count * i - duplicated if i > 0 else 0
    end = count * (i + 1) -duplicated if i > 0 else count
    print(f'开始绘制第 {i+1} / {n} 幅图片 ...')
    plot_theory_of_chan_2(
        df_origin=df_ohlc[start:end],
        df_merged=merged_candle_list,
        file_path=filename,
        merged_line_width=2,
        x=6400,
        y=4800,
        dpi=288
    )
    print(f'图片已保存在 {filename} 。')

time_end = dt.datetime.now()
print(f'全部完成，总用时： {time_end - time_start}')

开始绘制第 1 / 1 幅图片 ...


NameError: name 'df_merged' is not defined